# Guidelines for Prompting

This guide is designed to introduce and explain two key principles for crafting effective prompts for large language models. It also details the tactics associated with these principles. Upon learning these concepts, we will implement these techniques to develop a chatbot capable of processing orders from various restaurants.

The two principles we will explore are:
1. Write clear and specific instructions
2. Give the model time to "think"

## Setup
#### Load the API key and relevant Python libaries.

In [162]:
#### Google Generative AI ####
import google.generativeai as genai

#### General ####
import os
from IPython.display import display, HTML, Markdown
from redlines import Redlines

#### UI ####
import panel as pn

In [173]:
# Setup the API_KEY and display environment
genai.configure(api_key=os.getenv('API_KEY'))
pn.extension()
panels = []

# Configure the model 
model = genai.GenerativeModel(
            model_name='gemini-1.5-pro-latest'

#### Helper Function
Throughout this notebook, we will use Google's `gemini-1.5-pro-latest` model and the [generate content API](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content). 

This helper function will make it easier to use prompts and look at the generated outputs. This helper function is also used to create a chatbot at the end of the notebook, and hence is capable of multi-turn conversations as well.

In [ ]:
def get_response(prompt, messages=None, temperature=0):
    if messages == None:
        messages = []
    
    messages.append({'role': 'user', 'parts': [prompt]})
    
    # Set up the model
    generation_config = {
      "temperature": temperature,
      "top_p": 1,
      "top_k": 1,
      "max_output_tokens": 2048,
    }
    
    # Generate content based on user prompt
    response = model.generate_content(
                contents=messages,
                generation_config=generation_config,
            )
    response_dict = {
        'role': response.candidates[0].content.role,
        'parts': [response.candidates[0].content.parts[0].text]
    }
    messages.append(response_dict)
    
    return messages

## Prompting Principles
- **Principle 1: Write clear and specific instructions**

### Clear and Specific Instructions

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [33]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Provide explicit and specific instructions to guide models towards the desired output, avoiding irrelevant responses by utilizing longer prompts for more context and clarity.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [35]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""

response = get_response(prompt)
print(response[1]['parts'][0])

```json
[
  {
    "book_id": 1,
    "title": "The Chronicles of the Azure Knight",
    "author": "Aetheria Nightshade",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Quantum Enigma",
    "author": "Orion Nebula",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "Whispers of the Ancient Forest",
    "author": "Emeralda Willowbrook",
    "genre": "Nature Writing"
  }
]
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [37]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_response(prompt)
print("Completion for Text 1:")
print(response[1]['parts'][0])

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the teabag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - Take out the tea bag.
Step 6 - Add some sugar or milk to taste (optional).


In [38]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_response(prompt)
print("Completion for Text 2:")
print(response[1]['parts'][0])

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [39]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

response = get_response(prompt)
print(response[1]['parts'][0])

<grandparent>: The willow bends with the wind, yet remains rooted in the earth; the waves crash against the shore, but the sand endures; the stars twinkle amidst the darkness.


### Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [40]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_response(prompt_1)
print("Completion for prompt 1:")
print(response[1]['parts'][0])

Completion for prompt 1:
### 1 - Summary ###
In a charming village, siblings Jack and Jill embarked on an adventure to fetch water but faced a mishap along the way.

### 2 - French Translation ###
Dans un charmant village, les frères et sœurs Jack et Jill sont partis chercher de l'eau au puits situé au sommet d'une colline. Pendant leur ascension, ils chantaient joyeusement, mais le malheur a frappé - Jack a trébuché sur une pierre et a dégringolé la colline, suivi par Jill. Bien qu'ils soient légèrement amochés, ils sont rentrés chez eux pour se réconforter dans leurs bras. Malgré l'incident, leur esprit aventureux est resté intact et ils ont continué à explorer avec plaisir.

### 3 - Names in French Summary ###
* Jack
* Jill

### 4 - JSON Object ###
```
{
  "french_summary": "Dans un charmant village, les frères et sœurs Jack et Jill sont partis chercher de l'eau au puits situé au sommet d'une colline. Pendant leur ascension, ils chantaient joyeusement, mais le malheur a frappé - Jac

**Ask for output in a specified format**

In [44]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_response(prompt_2)
print("Completion for prompt 2:\n")
print(response[1]['parts'][0])

Completion for prompt 2:

Summary: Jack and Jill, siblings from a charming village, embarked on a quest to fetch water but encountered a mishap on their way.
Translation: Jack et Jill, deux frères et sœurs d'un charmant village, se sont lancés dans une quête pour aller chercher de l'eau mais ont rencontré un incident sur leur chemin.
Names: 
- Jack
- Jill
Output JSON: {
  "french_summary": "Jack et Jill, deux frères et sœurs d'un charmant village, se sont lancés dans une quête pour aller chercher de l'eau mais ont rencontré un incident sur leur chemin.",
  "num_names": 2
}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [45]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

response = get_response(prompt)
print(response[1]['parts'][0])

The student's solution is correct.


**Note that the student's solution is actually not correct.**

**We can fix this by instructing the model to work out its own solution first.**

In [46]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


## Model Limitations: Hallucinations
- Boie is a real company, however the product name is not real.

In [47]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""

response = get_response(prompt)
print(response)

**AeroGlide UltraSlim Smart Toothbrush by Boie**

The AeroGlide UltraSlim Smart Toothbrush is a cutting-edge oral hygiene device that combines advanced technology with exceptional design.

**Features:**

* **Supersonic Motor:** Generates up to 48,000 vibrations per minute for unparalleled plaque removal and efficient cleaning.
* **Ultra-Slim Brush Head:** Designed to reach and clean even the most difficult-to-reach areas, like back molars.
* **Pressure Sensor:** Monitors brushing pressure and alerts you if you're brushing too hard to prevent gum damage.
* **Bluetooth Connectivity:** Pairs with the Boie app for real-time brushing data, personalized guidance, and reminders.
* **Customized Brushing Modes:** Six pre-programmed modes (Clean, Sensitive, Massage, Polish, Deep Clean, Tongue Care) to tailor your brushing experience.
* **Quad-Zone Timer:** Guides you through a thorough 2-minute brushing session, dividing your mouth into four zones for even coverage.
* **Quad-Pacer Interval Timer

## Iterative Prompt Development
Iteratively analyze and refine prompts to generate marketing copy from a product fact sheet.

### Generate a marketing product description from a product fact sheet

In [48]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [49]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Indulge in mid-century sophistication with our exceptional SWC office chair. Its sleek design seamlessly integrates into both home and business environments, exuding timeless elegance. With a vast selection of shell colors and base finishes, you can tailor it to your unique aesthetics.

Choose from plastic back and front upholstery (SWC-100) or enveloping full upholstery (SWC-110) in a spectrum of 10 fabrics and 6 leathers. Whether you prefer stainless steel, matte black, gloss white, or chrome, our versatile base options complement your décor flawlessly.

Crafted in Italy, this chair prioritizes both comfort and durability. Its pneumatic adjustment ensures effortless height customization, while its 5-wheel plastic coated aluminum base glides smoothly across surfaces. Select soft or hard-floor casters for optimal maneuverability.

Customize your seating experience with two foam density options – medium (1.8 lb/ft3) or high (2.8 lb/ft3) – and choose from armless or 8-position PU armrest

#### Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [64]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Step back in time with our stunning mid-century-inspired chair. Its customizable design, from shell color to base finishes and upholstery options, makes it perfect for both home and business settings. With its durable aluminum base and adjustable seat, comfort and style go hand-in-hand.


In [65]:
len(response[1]['parts'][0].split(" "))

43

#### Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [66]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Sturdy and stylish, this mid-century modern chair features a durable cast aluminum shell with nylon coating and a plush HD36 foam seat. Its 5-wheel plastic coated aluminum base provides effortless mobility, while various finish and upholstery options allow for customization to suit any décor.


In [67]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

This mid-century modern office chair boasts a durable construction featuring a cast aluminum shell base with a modified nylon coating. Its comfortable HD36 foam seat is supported by a sturdy 5-wheel plastic coated aluminum base with your choice of caster options.

Product IDs: SWC-100, SWC-110


#### Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [74]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format the product description, product ID, and the table 'Product Dimensions' 
as HTML that can be used in a website. Place all the elements in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

```html
<div>
  <h1>Mid-Century Executive Office Chair</h1>

  <p>This office chair is part of a collection of mid-century inspired office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. It is available in several options of shell color and base finishes, and with plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options are stainless steel, matte black, gloss white, or chrome. The chair is available with or without armrests, and is suitable for home or business settings, and is qualified for contract use.</p>

  <p><strong>Construction:</strong></p>

  <ul>
    <li>5-wheel plastic coated aluminum base</li>
    <li>Pneumatic chair adjust for easy raise/lower action</li>
    <li>HD36 foam seat</li>
  </ul>

  <p><strong>Materials:</strong></p>

  <ul>
    <li>SHELL BASE GLIDER: Cast Aluminum with modified nylon PA6/PA66 coating. Shell thickness: 10 mm.</li>
  </ul>

  <h2>Product ID</h2>


In [75]:
display(HTML(response[1]['parts'][0]))

Dimension,Measurement (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat height,17.32
Seat depth,16.14


## Summarizing
Summarize text with a focus on specific topics.

### Text to summarize

In [107]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

#### Summarize with a word/sentence/character limit

In [108]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

***Soft and adorable panda plush toy with a friendly face, but slightly overpriced for its size. Arrived promptly.**


#### Summarize with a focus on shipping and delivery

In [110]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

***Arrived a day earlier than expected, allowing for pre-gifting playtime.***


#### Summarize with a focus on price and value

In [115]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

### Item overpriced; consider larger size option for same price.


In [116]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Package arrived a day earlier than expected.


#### Summarize multiple product reviews

In [117]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [126]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """
    
    response = get_response(prompt)
    print(i, response[1]['parts'][0], "\n")

0 ***Cute and soft panda plush toy, perfect for gifting, but may be overpriced for its size.*** 

1 ***Lamp arrived quickly, had missing parts but company quickly resolved the issue, indicating excellent customer service.*** 

2 Electric toothbrush with impressive battery life but a small toothbrush head and expensive replacement heads. 

3 ### Review Summary: 
Product quality declined, but still a good value during seasonal sales. Motor issues after a year. 



## Inferring

Infer sentiment and topics from product reviews and news articles.

### Product review text

In [127]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

#### Sentiment (positive/negative)

In [131]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

Positive


In [133]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

positive


#### Identify types of emotions

In [134]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

- contented
- grateful
- satisfied
- relieved
- pleased


#### Identify anger

In [136]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

no


#### Extract product and company name from customer reviews

In [138]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

```json
{
  "Item": "lamp",
  "Brand": "Lumina"
}
```


#### Doing multiple tasks at once

In [140]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```


#### Inferring topics

In [141]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

#### Infer 5 topics

In [143]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

- Employee satisfaction
- NASA
- Social Security Administration
- Work environment
- Government


In [145]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

#### Make a news alert for certain topics

In [146]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_response(prompt)
print(response[1]['parts'][0])

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [147]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response[1]['parts'][0].split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


## Transforming

Use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

### Translation

Gemini is trained with sources in many languages. This gives the model the ability to do translation.

In [148]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""

response = get_response(prompt)
print(response[1]['parts'][0])

Hola, me gustaría pedir una licuadora


In [149]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_response(prompt)
print(response[1]['parts'][0])

French


In [150]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_response(prompt)
print(response[1]['parts'][0])

**French:** J'aimerais commander un ballon de basket.

**Spanish:** Me gustaría pedir un balón de baloncesto.

**English pirate:** Ahoy there, matey! I be wantin' to order meself a basketball.


In [151]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""

response = get_response(prompt)
print(response[1]['parts'][0])

**Formal:**
¿Le gustaría ordenar una almohada?

**Informal:**
¿Quieres pedirte una almohada?


#### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [152]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
]

In [153]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_response(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_response(prompt)
    print(response[1]['parts'][0], "\n")

Original message (French): La performance du système est plus lente que d'habitude.
**English:**
System performance is slower than usual.

**Korean:**
시스템 성능이 평소보다 느립니다. 

Original message (Spanish): Mi monitor tiene píxeles que no se iluminan.
**English:** My monitor has pixels that don't light up.

**Korean:** 내 모니터에 불이 들어오지 않는 픽셀이 있습니다. 

Original message (Italian): Il mio mouse non funziona
**English:** My mouse is not working

**Korean:** 마우스가 작동하지 않습니다 

Original message (Polish): Mój klawisz Ctrl jest zepsuty
**English:** My Ctrl key is broken

**Korean:** 제 Ctrl 키가 고장났습니다 

Original message (Chinese): 我的屏幕在闪烁
**English:** My screen is flickering 

**Korean:** 화면이 깜빡입니다. 



#### Tone Transformation
Writing can vary based on the intended audience. Gemini can produce different tones.

In [154]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_response(prompt)
print(response[1]['parts'][0])

Greetings [Name],

My name is Joe and I would like to share the attached specification for a standing lamp that I believe would be a valuable addition to your product line.


#### Format Conversion
Gemini can translate between formats. The prompt should describe the input and output formats.

In [155]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_response(prompt)
print(response[1]['parts'][0])

```html
<h2>Resturant Employees</h2>
<table>
<thead>
<tr>
<th>Name</th>
<th>Email</th>
</tr>
</thead>
<tbody>
<tr>
<td>Shyam</td>
<td>shyamjaiswal@gmail.com</td>
</tr>
<tr>
<td>Bob</td>
<td>bob32@gmail.com</td>
</tr>
<tr>
<td>Jai</td>
<td>jai87@gmail.com</td>
</tr>
</tbody>
</table>
```


In [156]:
display(HTML(response[1]['parts'][0]))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


#### Spellcheck/Grammar check.

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'

In [157]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_response(prompt)
    print(response[1]['parts'][0])

The girl with the black and white puppies has a ball.
No errors found.
No errors found
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
**Corrected text:**

That medicine affects my ability to sleep. Have you heard of the butterfly effect?
No errors found


In [170]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"Proofread and write a grammatically correct version of this review: ```{text}```"
response = get_response(prompt)
print(response[1]['parts'][0])

I purchased this panda for my daughter's birthday because she frequently takes mine from my room. Adults appreciate pandas too! She carries it with her everywhere because it is exceptionally soft and adorable.

One ear is slightly lower than the other, which I do not believe was intended to be asymmetrical. However, considering the price, it is somewhat small. I believe there may be larger alternatives available for the same price.

To my delight, it arrived a day earlier than anticipated, allowing me to enjoy it myself before presenting it to my daughter.


In [171]:
diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I purchased </span>this <span style='color:green;font-weight:700;'>panda </span>for my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for her </span><span style='color:green;font-weight:700;'>daughter's </span>birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she <span style='color:red;font-weight:700;text-decoration:line-through;'>keeps taking </span><span style='color:green;font-weight:700;'>frequently takes </span>mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  Yes, adults also like </span><span style='color:green;font-weight:700;'>room. Adults appreciate </span>pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too! </span>She <span style='color:red;font-weight:700;text-decoration:line-through;'>takes </span><span style='color:green;font-weight:700;'>carries </span>it <span style='color:green;font-weight:700;'>with her </span>everywhere <span style='color:red;font-weight:700;text-decoration:line-through;'>with her, and it's super </span><span style='color:green;font-weight:700;'>because it is exceptionally </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>adorable. </span>

<span style='color:green;font-weight:700;'></span>One <span style='color:red;font-weight:700;text-decoration:line-through;'>of the ears </span><span style='color:green;font-weight:700;'>ear </span>is <span style='color:red;font-weight:700;text-decoration:line-through;'>a bit </span><span style='color:green;font-weight:700;'>slightly </span>lower than the other, <span style='color:red;font-weight:700;text-decoration:line-through;'>and </span><span style='color:green;font-weight:700;'>which </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>don't think that </span><span style='color:green;font-weight:700;'>do not believe </span>was <span style='color:red;font-weight:700;text-decoration:line-through;'>designed </span><span style='color:green;font-weight:700;'>intended </span>to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's a bit small for what </span><span style='color:green;font-weight:700;'>However, considering the price, it is somewhat small. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>paid for it though. I think </span><span style='color:green;font-weight:700;'>believe </span>there <span style='color:red;font-weight:700;text-decoration:line-through;'>might </span><span style='color:green;font-weight:700;'>may </span>be <span style='color:red;font-weight:700;text-decoration:line-through;'>other options that are bigger </span><span style='color:green;font-weight:700;'>larger alternatives available </span>for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price. </span>

<span style='color:green;font-weight:700;'>To my delight, it </span>arrived a day earlier than <span style='color:red;font-weight:700;text-decoration:line-through;'>expected, so I got </span><span style='color:green;font-weight:700;'>anticipated, allowing me </span>to <span style='color:red;font-weight:700;text-decoration:line-through;'>play with </span><span style='color:green;font-weight:700;'>enjoy </span>it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>presenting </span>it to my daughter.

In [164]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_response(prompt)
display(Markdown(response[1]['parts'][0]))

## Review of the Cute and Fluffy Panda Plushie

**Product:** Panda Plushie

**Intended Audience:** Adults and children

**Positive Features:**

* Unparalleled softness and cuddly texture
* Adorable and captivating panda design
* Conveniently portable, accompanying the user everywhere

**Minor Criticism:**

* Asymmetrical ears, with one ear positioned lower than the other (likely a design flaw)
* Size discrepancy compared to the price, with larger options available at similar costs

**Exceptional Delivery:**

* Swift arrival, exceeding the expected delivery date

**Overall Impression:**

This panda plushie is a delightful companion, boasting exceptional softness and an endearing design. Its portability allows for constant companionship, bringing joy to both adults and children alike. While the asymmetrical ears may be a minor aesthetic flaw, the overall cuteness and undeniable cuddle factor more than compensate.

Despite its slightly smaller size compared to other options on the market, the panda plushie offers an exceptional quality-to-price ratio. The prompt delivery also ensures timely gratification.

**Recommendation:**

Highly recommended for anyone seeking a loyal and lovable plushie companion. Its irresistibly soft texture, charming design, and convenient portability make it an ideal choice for those who cherish comfort and cuteness.

## Expanding

Generate customer service emails that are tailored to each customer's review.

In [175]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

#### Customize the automated reply to a customer email

In [176]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_response(prompt)
print(response[1]['parts'][0])

Dear Valued Customer,

Thank you for taking the time to share your feedback on our product. We appreciate your honesty and understand your concerns regarding the price increase and perceived decline in quality.

We apologize for any inconvenience or disappointment this may have caused. We value your loyalty and are committed to providing high-quality products. We will take your feedback into consideration as we work to improve our offerings.

If you have any further concerns or would like to discuss this matter further, please do not hesitate to reach out to our customer service team.

Thank you for your continued support.

Sincerely,
AI Customer Agent


#### Remind the model to use details from the customer's email

In [178]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_response(prompt, temperature=0.7)
print(response[1]['parts'][0])

Dear Valued Customer,

Thank you for taking the time to provide us with your feedback. We appreciate your continued patronage.

We sincerely regret to hear that you have experienced some concerns with the product's base and motor noise. We understand that these issues can be frustrating.

As you mentioned, the warranty for your previous system has expired. However, we encourage you to reach out to our customer service team directly. They will be happy to explore any possible options or solutions with you.

We value your loyalty and strive to provide our customers with high-quality products and support. Please do not hesitate to contact us if you have any further concerns or questions.

Sincerely,
AI customer agent


## Multi Turn Conversations

You can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

### ChefCurry
We can automate the collection of user prompts and assistant responses to build a  order bot. The Order Bot will take orders from all restaurants at a particular location at the university (for e.g. Curry Student Center). 

In [ ]:
# Configure the model 
model = genai.GenerativeModel(
            model_name='gemini-1.5-pro-latest',
            system_instruction="""
You are ChefCurry, an automated service to collect orders for the following 4 restaurants: \
1. Sweet Tomatoes, 2. Kigo Kitchen, 3. Popeye’s, 4. Starbucks. \
You must perform the following steps in the same order: \
Step 1: You first greet the customer with your name, and mention your \
functionality, which is to collect orders for any of the 4 restaurants by asking \
which restaurant the user would like to order from. \
Step 2: Based on the restaurant chosen, provide the menu overview and \
collect the order. \
Step 3: You wait to collect the entire order, then summarize it and \
check for a final time if the customer wants to add anything else. \
Step 4: Finally collect the payment. \
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The Starbucks menu includes \
Coffee: \
Americano 2.95 (Tall), 3.45 (Grande), 3.95 (Venti) \
Caffe Latte 3.65 (Tall), 4.15 (Grande), 4.65 (Venti) \
Tea: \
Chai Tea Latte 3.45 (Tall), 4.45 (Grande), 4.95 (Venti) \
Green Tea Latte 3.45 (Tall), 4.45 (Grande), 4.95 (Venti) \
Food \
Blueberry Muffin 2.95 \
Chocolate Croissant 3.45 \
The Sweet Tomatoes menu includes \
Sweet Tomatoes Pie 17.35 (Medium) 21.95 (Large) \
Slices: \
Cheese 3.75 \
Pepperoni 4.50 \
The Kigo Kitchen menu includes \
Bowls: \
Teriyaki Chicken Bowl 8.99 \
Spicy Pork Bowl 9.49 \
Noodles \
Soba Noodle Salad 7.99 \
Sides: \
Edamame 3.99 \
Gyoza 4.99 \
The Popeye's menu includes \
Chicken: \
3-Piece Chicken Tenders 5.99 \
Classic Chicken Sandwich 3.99 \
Sides
Cajun Fries 2.49 \
Red Beans and Rice 2.99 \
Desserts: \
Apple Pie 1.69 \
""")

In [ ]:
def collect_messages(_):
    global history
    prompt = text_input.value
    text_input.value = ''

    history = get_response(prompt, history, 0.7)
    response = history[len(history)-1]['parts'][0]
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)

In [ ]:
history = []
text_input = pn.widgets.TextInput(value='Hi', placeholder='Enter your text here...')
button = pn.widgets.Button(name='Chat!', button_type='primary')
chat_output = pn.bind(collect_messages, button)

chatbot_interface = pn.Column(
    text_input,
    pn.Row(button),
    pn.panel(chat_output, loading_indictor=True, height=300)
)

chatbot_interface   